In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('wiki_articles.csv', encoding='utf-8'  )

In [10]:
import csv , os, glob , random, time , codecs
import pandas as pd
import requests, urllib
from bs4 import BeautifulSoup
import time
import random

url_1 = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html'
url_2 = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html'
url_3 = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html'

# collect all of the urls in a list
url_list = []
for url in  [url_1 , url_2 , url_3]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    for link in soup.find_all('a'):
        url_list.append(link.get('href'))

In [19]:
df.to_csv('wiki_articles_with_urls.csv')

In [18]:
df['wili_urls'] = url_list

In [9]:
# make one text file of all of intro and plots

script =''
for i in df.index:
    script += ' '+' '.join( df.loc[i,['intro', 'plot']].dropna())

In [20]:
# Using this function to do the preprocessing and cleaning
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))

def CLEANING(string):
    string = string.lower() 
    tokenizer = RegexpTokenizer("[\w']+")  
    clean_text = tokenizer.tokenize(string)
    clean_words = []
    for word in clean_text:
        if word not in stop_words:
            clean_words.append(word)
    lemmatizer = WordNetLemmatizer() 
    lemmat = [lemmatizer.lemmatize(wor) for wor in clean_words]
    ps = PorterStemmer()
    bagofword = [ps.stem(w) for w in lemmat]
    return bagofword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
bagofword = CLEANING(script)
all_vocab = set(bagofword)

# term_id and vocab
dic_of_term_id = {}
dic_of_vocab = {}
for i,j in enumerate(all_vocab):
    num = 'term_' + str(i)
    dic_of_vocab[j] = num
    dic_of_term_id[num] = j

In [22]:
clean_df = pd.DataFrame()

for i in df.index:
    
    clean_df.loc[i,'text'] =' '.join(df[['intro', 'plot']].loc[i].dropna())

In [23]:
clean_df['text'] = clean_df['text'].map(CLEANING)

In [24]:
clean_df['unique_words'] = clean_df['text'].map(set)

In [25]:
df.to_csv('wiki_Cleaned_articles.csv')

In [34]:
no = []
vocabulary = {}
for i in clean_df.index:
    for word in clean_df.loc[i,'unique_words']:
        if dic_of_vocab[word] in vocabulary.keys():
            vocabulary[dic_of_vocab[word]].append('doc_' + str(i))
        elif dic_of_vocab[word] not in vocabulary.keys():
            vocabulary[dic_of_vocab[word]] = ['doc_' + str(i)]
        else:
            no.append(word)
            
    

2.1.2) Execute the query

In [80]:
# first search

from itertools import chain
from  collections import Counter

query = CLEANING(input('Please enter your: '))
answer = []
l = []
for q in query:
    answer += vocabulary[dic_of_vocab[q]]
    
#answer = set(chain.from_iterable(answer)) 
c = Counter(answer).most_common(len(query))
docs_list = [x for x,y in c]

for an in docs_list:
    l.append(int(an.split('_')[1]))
    
df.loc[l, ['title' , 'intro','wili_urls'] ]

Please enter your: disney movies 2019


,wili_urls,title,intro
11409,https://en.wikipedia.org/wiki/Men_Don%27t_Leave,Thumbelina (1994 film),Thumbelina (also known as Hans Christian Ander...
776,https://en.wikipedia.org/wiki/The_Delightful_R...,Hollywood Party (1934 film),"Hollywood Party, also known under its working ..."
2428,https://en.wikipedia.org/wiki/Penitentiary_(19...,Sullivan's Travels,Sullivan's Travels is a 1941 American comedy f...


2.2) Conjunctive query & Ranking score

In [113]:
def WORD_COUNT(word , doc):
    count = 0
    for w in doc:
        if w == word:
            count+=1              
    return count

def COSINE_SIMIL(s1, s2): 
    return np.dot(s1,s2)/ (np.sqrt(sum(s1**2)) * np.sqrt(sum(s2**2)))

In [104]:
#  Inverted index

no = []
inverted_index = {}
N = 30000

for term_id in vocabulary.keys():
    inverted_index[term_id] = []
    for doc_id in vocabulary[term_id]:
        term_frq = WORD_COUNT(dic_of_term_id[term_id] , clean_df.loc[int(doc_id.split('_')[1]) ,'text'])
        n = len(clean_df.loc[int(doc_id.split('_')[1]) , 'text'])
        
        inverted_index[term_id].append((doc_id, (term_frq /n )*np.log(N/len(vocabulary[term_id])))) 

In [ ]:
import pickle 

pickle.dump( inverted_index  , open( "inverted_index_in_PART.pkl", "wb"))
pickle.dump( vocabulary  , open( "vocabulary_in_PART.pkl", "wb" ))
pickle.dump( dic_of_term_id  , open( "dic_of_term_id_in_PART.pkl", "wb"))
pickle.dump( dic_of_vocab  , open( "dic_of_vocab_in_PART.pkl", "wb" ))

In [292]:
# second search

from itertools import chain
from  collections import Counter
def seach_2(query): 

    N= 30000

    query = set(CLEANING(input('Please enter your: ')))
    n = len(query)
    answer = []
    l = np.zeros(n)

    for o,q in enumerate(query):
        try:
            answer += vocabulary[dic_of_vocab[q]]
            l[o] = (WORD_COUNT(q ,query ) /n )*np.log(N/len(dic_of_vocab[q]))
        except:
            pass

    c = Counter(answer).most_common(7)
    docs_list = [x for x,y in c]



    sim= {} 
    for doc in docs_list:
        sim[doc] = np.zeros(n)
        for o,q in enumerate(query):
            try:
                sim[doc][o] = [b for a,b in inverted_index[dic_of_vocab[q]] if a == doc][0]
            except:
                pass   


    similarity = {}           
    for s in sim.keys():
        similarity[s] = COSINE_SIMIL(l , sim[s] ) 

    doc_num = sorted(similarity.items(), key=lambda x: x[1] , reverse=True)

    table = df.loc[[int(e.split('_')[1]) for e,r in doc_num], ['title' , 'intro','wili_urls' , 'Similarity'] ]
    table['Similarity'] = [r for e,r in doc_num]

    return table[['title' , 'intro','wili_urls' , 'Similarity'] ]

In [389]:
seach_2(query)

Please enter your: love sad romance Claire Danes


,title,intro,wili_urls,Similarity
13330,The Hours (film),The Hours is a 2002 psychological drama film d...,https://en.wikipedia.org/wiki/American_Beauty_...,0.878592
24,Song of the Flame,Song of the Flame is a 1930 pre-Code musical f...,https://en.wikipedia.org/wiki/The_Lure_of_the_...,0.740995
1807,Ninotchka,Ninotchka is a 1939 American comedy film made...,https://en.wikipedia.org/wiki/The_Witching_Hou...,0.695487
382,The Match King,The Match King is a 1932 American Pre-Code dra...,https://en.wikipedia.org/wiki/The_High_Sign,0.695487
378,Man Wanted,Man Wanted is a pre-Code 1932 romance film sta...,https://en.wikipedia.org/wiki/The_Four_Horseme...,0.681716
15339,Letters to Juliet,Letters to Juliet is a 2010 American romantic ...,https://en.wikipedia.org/wiki/Deception_(2008_...,0.544406
3995,Blonde Ice,Blonde Ice is a crime film noir movie released...,https://en.wikipedia.org/wiki/Fashion_Model_(f...,0.468203


3. Define a new score!

In [182]:
import heapq

In [202]:
columnn = ['title', 'Directed by', 'Produced by', 'Written by','Screenplay by', 'Story by', 'Based on', 'Starring', 
 'Narrated by', 'Music by', 'Cinematography', 'Edited by', 'Production', 'company','Distributed by', 'Release date', 
 'Running time', 'Country', 'Language','Budget', 'Box', 'office']

In [203]:
for i in  df.index:
    
    clean_df.loc[i,'more_info'] =' '.join(df[columnn].loc[i].dropna())

In [200]:
clean_df.to_csv('clean_wiki_articles_with_urls.csv')

In [257]:
import heapq

def jaccard(s1, s2):
    s1 ,s2 = set(s1), set(s2) 
    intersect = s1.intersection(s2)
    return len(intersect)/(len(s1.union(s2)))

def Euclidean(s1,s2):
    return np.linalg.norm(s1-s2)

# reasult_jaccard = []
# reasult_Euclidean = []
# heapq.heapify(reasult_Euclidean)
# heapq.heapify(reasult_jaccard)

# for ii in table.index:
#     heapq.heappush(reasult_jaccard ,( jaccard(clean_df.loc[ii , 'more_info'], query , ii )))
#     heapq.heappush(reasult_Euclidean ,( Euclidean(clean_df.loc[ii , 'more_info'], query , ii )))
        

In [231]:
clean_df['more_info'] = clean_df['more_info'].map(CLEANING)

In [252]:
Euclidean(clean_df.loc[676,'more_info'], ['sad', 'love', 'william'])

NameError: name 'numpy' is not defined

In [390]:
# Third search by heapq
import heapq 
from itertools import chain
from  collections import Counter
N= 30000

query = set(CLEANING(input('Please enter your: ')))
n = len(query)
answer = []
l = np.zeros(n)

for o,q in enumerate(query):
    try:
        answer += vocabulary[dic_of_vocab[q]]
        l[o] = (WORD_COUNT(q ,query ) /n )*np.log(N/len(dic_of_vocab[q]))
    except:
        pass

c = Counter(answer).most_common(7)
docs_list = [x for x,y in c]


sim= {} 
for doc in docs_list:
    sim[doc] = np.zeros(n)
    for o,q in enumerate(query):
        try:
            sim[doc][o] = [b for a,b in inverted_index[dic_of_vocab[q]] if a == doc][0]
        except:
            pass   


similarity = {}           
for s in sim.keys():
    similarity[s] = COSINE_SIMIL(l , sim[s] ) 

doc_num = sorted(similarity.items(), key=lambda x: x[1] , reverse=True)

table = df.loc[[int(e.split('_')[1]) for e,r in doc_num], ['title' , 'intro','wili_urls' , 'Similarity'] ]
table['Similarity'] = [r for e,r in doc_num]

df_search= table[['title' , 'intro','wili_urls' , 'Similarity'] ]


Please enter your: love sad romance Claire Danes


In [391]:
import heapq
seach_3_col = ['title' , 'intro','wili_urls' , 'Similarity']
search_3 = []
list_info = []
heapq.heapify(search_3)

for wi in query:
    list_info += inverted_index_moreinfo[wi]
    
    
cn = Counter(list_info ).most_common(15)
docs_list_info = [x for x,y in c]    

for ii in docs_list_info:
        heapq.heappush(search_3 , (10*jaccard(query , clean_df.loc[int(ii.split('_')[1]),'more_info']) ,ii))

jac_info =  heapq.nlargest(5, search_3)       
jac_simil = [int(e.split('_')[1] )for r,e in jac_info]

indx_seach3 =[]
for cod in jac_simil:
    if cod in df_search.index:
        df_search.loc[cod, 'Similarity'] += float(*[f for f,g in jac_info if int(g.split('_')[1]) == cod])
    else:
        indx_seach3.append(cod)
        
        
df_search = pd.concat([ df_search, df.loc[indx_seach3, seach_3_col]], axis=0 )
df_search.loc[ indx_seach3, 'Similarity'] =  [f for f,g in jac_info if int(g.split('_')[1]) in indx_seach3 ]                      

df_search.sort_values('Similarity',ascending=False)

,title,intro,wili_urls,Similarity
13330,The Hours (film),The Hours is a 2002 psychological drama film d...,https://en.wikipedia.org/wiki/American_Beauty_...,0.878592
24,Song of the Flame,Song of the Flame is a 1930 pre-Code musical f...,https://en.wikipedia.org/wiki/The_Lure_of_the_...,0.949328
1807,Ninotchka,Ninotchka is a 1939 American comedy film made...,https://en.wikipedia.org/wiki/The_Witching_Hou...,0.922760
382,The Match King,The Match King is a 1932 American Pre-Code dra...,https://en.wikipedia.org/wiki/The_High_Sign,0.695487
378,Man Wanted,Man Wanted is a pre-Code 1932 romance film sta...,https://en.wikipedia.org/wiki/The_Four_Horseme...,0.681716
15339,Letters to Juliet,Letters to Juliet is a 2010 American romantic ...,https://en.wikipedia.org/wiki/Deception_(2008_...,0.544406
3995,Blonde Ice,Blonde Ice is a crime film noir movie released...,https://en.wikipedia.org/wiki/Fashion_Model_(f...,0.468203


In [396]:
df_search.sort_values('Similarity',ascending=False)

,title,intro,wili_urls,Similarity
24,Song of the Flame,Song of the Flame is a 1930 pre-Code musical f...,https://en.wikipedia.org/wiki/The_Lure_of_the_...,0.949328
1807,Ninotchka,Ninotchka is a 1939 American comedy film made...,https://en.wikipedia.org/wiki/The_Witching_Hou...,0.922760
13330,The Hours (film),The Hours is a 2002 psychological drama film d...,https://en.wikipedia.org/wiki/American_Beauty_...,0.878592
382,The Match King,The Match King is a 1932 American Pre-Code dra...,https://en.wikipedia.org/wiki/The_High_Sign,0.695487
378,Man Wanted,Man Wanted is a pre-Code 1932 romance film sta...,https://en.wikipedia.org/wiki/The_Four_Horseme...,0.681716
15339,Letters to Juliet,Letters to Juliet is a 2010 American romantic ...,https://en.wikipedia.org/wiki/Deception_(2008_...,0.544406
3995,Blonde Ice,Blonde Ice is a crime film noir movie released...,https://en.wikipedia.org/wiki/Fashion_Model_(f...,0.468203


In [380]:
df_search.loc[cod , seach_3_col]

title                                                Man Wanted
intro         Man Wanted is a pre-Code 1932 romance film sta...
wili_urls     https://en.wikipedia.org/wiki/The_Four_Horseme...
Similarity                                             0.681716
Name: 378, dtype: object

In [317]:
df_search 

,title,intro,wili_urls,Similarity
13330,The Hours (film),The Hours is a 2002 psychological drama film d...,https://en.wikipedia.org/wiki/American_Beauty_...,0.878592
24,Song of the Flame,Song of the Flame is a 1930 pre-Code musical f...,https://en.wikipedia.org/wiki/The_Lure_of_the_...,0.740995
1807,Ninotchka,Ninotchka is a 1939 American comedy film made...,https://en.wikipedia.org/wiki/The_Witching_Hou...,0.695487
382,The Match King,The Match King is a 1932 American Pre-Code dra...,https://en.wikipedia.org/wiki/The_High_Sign,0.695487
378,Man Wanted,Man Wanted is a pre-Code 1932 romance film sta...,https://en.wikipedia.org/wiki/The_Four_Horseme...,0.681716
15339,Letters to Juliet,Letters to Juliet is a 2010 American romantic ...,https://en.wikipedia.org/wiki/Deception_(2008_...,0.544406
3995,Blonde Ice,Blonde Ice is a crime film noir movie released...,https://en.wikipedia.org/wiki/Fashion_Model_(f...,0.468203


In [ ]:
love sad romance Claire Danes

In [ ]:
sim_info = {} 
for do in docs_list_info:
    sim_info[do] = np.zeros(n)
    for oi,qi in enumerate(query):
        try:
           sim_info[doc][oi] = [b for a,b in inverted_index_moreinfo[qi] if a == do][0]
        except:
            pass
                       
similarity_info = {}           
for ss in sim.keys():
    similarity_info[ss] = COSINE_SIMIL(l , sim_info[ss] )                        
                       
doc_infom = sorted(similarity_info.items(), key=lambda x: x[1] , reverse=True) 

In [333]:

no = []
inverted_index_moreinfo = {}
N = 30000

for term in vocabulary_moreinfo.keys():
    inverted_index_moreinfo[term] = []
    for doc_id in vocabulary_moreinfo[term]:
        term_frqin = WORD_COUNT(term , clean_df.loc[int(doc_id.split('_')[1]) ,'more_info'])
        n = len(clean_df.loc[int(doc_id.split('_')[1]) , 'more_info'])

        inverted_index_moreinfo[term].append((doc_id, (term_frqin /n )*np.log(N/len(vocabulary_moreinfo[term])))) 

In [334]:
inverted_index_moreinfo['love']

[('doc_0', 0.30719958916322754),
 ('doc_25', 0.13771016065937786),
 ('doc_82', 0.09508558712195136),
 ('doc_105', 0.08681727519830343),
 ('doc_236', 0.13771016065937786),
 ('doc_248', 0.1050945962926831),
 ('doc_258', 0.09740474778346239),
 ('doc_368', 0.20479972610881833),
 ('doc_368', 0.20479972610881833),
 ('doc_370', 0.1663997774634149),
 ('doc_371', 0.09287429439818506),
 ('doc_541', 0.11745866644476347),
 ('doc_568', 0.08681727519830343),
 ('doc_583', 0.09983986647804895),
 ('doc_607', 0.1050945962926831),
 ('doc_747', 0.07830577762984231),
 ('doc_817', 0.13771016065937786),
 ('doc_844', 0.08874654798048795),
 ('doc_852', 0.08681727519830343),
 ('doc_880', 0.09740474778346239),
 ('doc_1022', 0.09508558712195136),
 ('doc_1023', 0.06339039141463425),
 ('doc_1179', 0.11410270454634165),
 ('doc_1200', 0.08874654798048795),
 ('doc_1207', 0.07679989729080688),
 ('doc_1208', 0.12101801997339266),
 ('doc_1209', 0.08497009913025441),
 ('doc_1222', 0.09508558712195136),
 ('doc_1223', 0.054

In [330]:
no = []
more_info_unq = set(script_more_info)
vocabulary_moreinfo = {}
for i in clean_df.index:
    for word in clean_df.loc[i,'more_info']:
        if word in vocabulary_moreinfo.keys():
            vocabulary_moreinfo[word].append('doc_' + str(i))
        elif word not in vocabulary_moreinfo.keys():
            vocabulary_moreinfo[word] = ['doc_' + str(i)]
        else:
            no.append(word)

In [ ]:
vocabulary_moreinfo['raghumuktha']

In [322]:
script_more_info =[]
for i in clean_df.index:
    script_more_info +=clean_df.loc[i,'more_info']
    

In [327]:
dic_more_info = {}
dic_more_info_rev ={}
for i, j in enumerate(set(script_more_info)):
   dic_more_info['term'+str(i)] = j
   dic_more_info_rev[j]=  ['term'+str(i)]                 

In [329]:
dic_more_info['yaowu']

KeyError: 'yaowu'